In [7]:
import os 
os.environ['OPENAI_API_KEY']=os.getenv('OPENAI_API_KEY')
from dotenv import load_dotenv
load_dotenv()

Python-dotenv could not parse statement starting at line 1
Python-dotenv could not parse statement starting at line 5


True

In [8]:
##1) data ingestion
# 1)text loader
from langchain_community.document_loaders import TextLoader
loader=TextLoader("speech.txt")
loader

In [9]:
loader.load()

[Document(page_content='\n**Title: The Role of Machine Learning in Modern Cyberbullying Detection**\n\n*Introduction:*\n\n"Good [morning/afternoon/evening] everyone,\n\nMy name is Abdul Samad, a passionate Data Scientist based in Kerala, India. Today, I am honored to present my work on \'The Role of Machine Learning in Modern Cyberbullying Detection.\' As someone deeply invested in the potential of artificial intelligence, I have dedicated a significant portion of my career to developing solutions that can make our online spaces safer and more respectful.\n\n*Background:*\n\nCyberbullying has become a pervasive issue in the digital age. With the rise of social media and online communication platforms, the need for effective detection and prevention strategies is more critical than ever. Traditional methods of addressing cyberbullying often fall short due to the sheer volume of content and the nuanced nature of online interactions.\n\n*Our Solution - CyberGuard:*\n\nIn response to this 

In [10]:
# 1)WebBaseLoader
import bs4
from langchain_community.document_loaders import WebBaseLoader

loader=WebBaseLoader(web_path=('https://lilianweng.github.io/posts/2023-06-23-agent/'),
                     bs_kwargs=dict(parse_only=bs4.SoupStrainer(
                         class_=("post-title","post-content","post-header")

                     )))
loader

In [11]:
loader.load()

[Document(page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistakes and refine them for future steps, thereby improving the quality of final re

In [12]:
# 3)PyPDFLoader
from langchain_community.document_loaders import PyPDFLoader
loader=PyPDFLoader(file_path='ml.pdf')
loader

In [13]:
docs=loader.load()
docs

[Document(page_content='Machine Learning  \n \nMachine learning is a subset of artificial intelligence (AI) that focuses on \ndeveloping computer systems capable of learning and improving from data \nwithout being explicitly programmed.  Machine learning is teaching computers to \nlearn from data and make decisions or predictions based on that learning.  \nMachine learning is widely used in various fields and applications  such as:  \n• Face Recognition  \n• Object Detection  \n• Chatbots  \n• Recommendation Systems  \n• Autonomous Vehicles  \n• Disease Diagnosis  \n• Fraud detection  \nAnd many more it is nowadays being used in almost all sectors including \nhealthcare, education, business, construction, astronomy, etc.  \n \nTypes of Machine Learning : \n \nThere are many types of machine learning but the most famous types are:  \n➢ Supervised learning  \n➢ Unsupervised learning  \n➢ Reinforcement learning  \n \n \n \n ', metadata={'source': 'ml.pdf', 'page': 0}),
 Document(page_cont

In [14]:
##2) transform
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=20)
documents=text_splitter.split_documents(docs)
documents[:5]

[Document(page_content='Machine Learning  \n \nMachine learning is a subset of artificial intelligence (AI) that focuses on \ndeveloping computer systems capable of learning and improving from data \nwithout being explicitly programmed.  Machine learning is teaching computers to \nlearn from data and make decisions or predictions based on that learning.  \nMachine learning is widely used in various fields and applications  such as:  \n• Face Recognition  \n• Object Detection  \n• Chatbots  \n• Recommendation Systems  \n• Autonomous Vehicles  \n• Disease Diagnosis  \n• Fraud detection  \nAnd many more it is nowadays being used in almost all sectors including \nhealthcare, education, business, construction, astronomy, etc.  \n \nTypes of Machine Learning : \n \nThere are many types of machine learning but the most famous types are:  \n➢ Supervised learning  \n➢ Unsupervised learning  \n➢ Reinforcement learning', metadata={'source': 'ml.pdf', 'page': 0}),
 Document(page_content="Supervise

In [24]:
##3) embedding
from langchain_openai.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings

In [25]:
##4) vector store
from langchain_community.vectorstores import Chroma
db = Chroma.from_documents(documents,OpenAIEmbeddings())

In [17]:
##5) method1 - retrive via similarity search
result=db.similarity_search('what is linear regression')
result[0].page_content

'independent variables (or features). It assumes that this relationship is \napproximately linear, meaning that changes in the independent variables \nhave a linear effect on the dependent variable.  \n➢ Simple Linear Regression:  \nIn simple linear regression, there is only one input feature.  \nY= wx+b  \nY= target variable  \nW=slope of straight line  \n X=input features  \n b= y-intercept  \n➢ Multiple Linear Regression:  \nIn multiple linear regression, there is more than one input feature.  \nY=w1x1+w2x2+w3x3+…. +b'

In [23]:
#-----------------------------------------------------------------------------------------
##6) method2- retrive via retriver

# 1)
# from langchain_community.llms import Ollama
# llm=Ollama(model="llama2")
from langchain_openai import ChatOpenAI
llm=ChatOpenAI()
# 2)
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context. 
Think step by step before providing a detailed answer. 
<context>
{context}
</context>
Question: {input}""")
# 3)
## Create Stuff Docment Chain
from langchain.chains.combine_documents import create_stuff_documents_chain
document_chain=create_stuff_documents_chain(llm,prompt)

# 4)
retriever=db.as_retriever()

# 5)
## create_retrieval_chain
from langchain.chains import create_retrieval_chain
retrieval_chain=create_retrieval_chain(retriever,document_chain)

response=retrieval_chain.invoke({"input":"what is linear regression"})
response

{'input': 'what is linear regression',
 'context': [Document(page_content='independent variables (or features). It assumes that this relationship is \napproximately linear, meaning that changes in the independent variables \nhave a linear effect on the dependent variable.  \n➢ Simple Linear Regression:  \nIn simple linear regression, there is only one input feature.  \nY= wx+b  \nY= target variable  \nW=slope of straight line  \n X=input features  \n b= y-intercept  \n➢ Multiple Linear Regression:  \nIn multiple linear regression, there is more than one input feature.  \nY=w1x1+w2x2+w3x3+…. +b', metadata={'page': 2, 'source': 'ml.pdf'}),
  Document(page_content="There are 2 common techniques used in recommendation systems explained \nbelow.  \nTechniques : \n❖ Collaborative Filtering:  \n \n“Linear regression is used to predict the ratings that a user might give to a \nmovie they haven't seen yet. The idea is to find a linear relationship \nbetween the features (such as user behavior, 